In [1]:
# 리밸런싱 
# out-of-sample에서 리밸런싱 우선
# 매달 리밸런싱 진행
    # 리밸런싱은 해당 기간 최적화 돌려서 나온 비중을 갖고 매달 진행.
    # 1. (out of sample 기간) 2023.05.01에 투자를 시작할 때 과거 5년치 데이터로 최적화 문제 풀고, 다음 한달간 수익률 보기. 
    # 그리고 2023.06.01에 시작할 때는 그때까지 데이터로 최적화 풀고, 
    # 나온 가중치로 리밸런싱 하고 한달간 수익률 보기… 
    # 반복
# 2. look back 기간을 1년, 2년, 등 여러 기간으로 설정하고 돌려보기.

In [20]:
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [21]:
# GPT 종목 (GPT weight)
stocks_15 = {
    "AAPL": 0.084, "AMZN": 0.074, "NVDA": 0.064, "JPM": 0.054, "PG": 0.054,
    "PFE": 0.064, "JNJ": 0.064, "KO": 0.054, "XOM": 0.064, "NEE": 0.074,
    "GOOGL": 0.084, "MSFT": 0.084, "TSLA": 0.074, "NKE": 0.054, "BAC": 0.054
}

stocks_30 = {
    "MSFT": 0.067, "AMZN": 0.067, "NVDA": 0.067, "AAPL": 0.067, "GOOGL": 0.067,
    "ADBE": 0.067, "JNJ": 0.05, "PFE": 0.05, "MRK": 0.05, "ABT": 0.05, 
    "PG": 0.05, "KO": 0.05, "JPM": 0.05, "GS": 0.05, "CAT": 0.025, 
    "CVX": 0.025, "XOM": 0.025, "BA": 0.025, "TSLA": 0.025, "NEE": 0.025, 
    "NKE": 0.005, "VZ": 0.005, "CRM": 0.005, "UNH": 0.005, "WMT": 0.005, 
    "QCOM": 0.005, "BAC": 0.005, "V": 0.005, "MCD": 0.005, "INTC": 0.005
}

stocks_45 = {
    "AAPL": 0.05, "GOOGL": 0.04, "MSFT": 0.04, "NVDA": 0.03, "AMD": 0.03, 
    "ORCL": 0.02, "CRM": 0.02, "INTC": 0.01, "CSCO": 0.01, "JPM": 0.04, 
    "GS": 0.03, "BAC": 0.03, "MS": 0.02, "AXP": 0.02, "C": 0.01,
    "JNJ": 0.03, "UNH": 0.03, "PFE": 0.02, "ABBV": 0.02, "TSLA": 0.02, 
    "AMGN": 0.02, "GILD": 0.01, "PG": 0.03, "KO": 0.03, "NKE": 0.02, 
    "PEP": 0.02, "COST": 0.02, "WMT": 0.02, "TGT": 0.01, "XOM": 0.025, 
    "CVX": 0.025, "NEE": 0.02, "DUK": 0.01, "SO": 0.01, "SLB": 0.01,
    "MMM": 0.02, "CAT": 0.02, "HON": 0.02, "GE": 0.02, "ADP": 0.02,
    "AMZN": 0.02, "META": 0.02, "HD": 0.02, "VZ": 0.01, "MRK": 0.01
}

In [22]:
tickers_15 = list(stocks_15.keys())
tickers_30 = list(stocks_30.keys())
tickers_45 = list(stocks_45.keys())

### GMV + EPO weight

In [23]:
gmv_epo_15_weights = [0.049605485006088634, 0.07849495425750663, 0.06333929068001071, 0.05700662407849795, 0.07650289062642221, 0.05900628444525451, 0.057323863755786, 0.0475050526473884, 0.06851785110258568, 0.0551095984517666, 0.0635746953436858, 0.08684669491598282, 0.06555603211482636, 0.08160761317658853, 0.090003069397609]
gmv_epo_30_weights = [0.016666779461997225, 0.028582382826203727, 0.03181481366190954, 0.04612454337672658, 0.03430208600932986, 0.02496457702197057, 0.034559689579773685, 0.03603933972602465, 0.0348256115466567, 0.02719368909866466, 0.01666666768426058, 0.03550281897656019, 0.03316982644195968, 0.02106295226563651, 0.019372147251594442, 0.01873074735009172, 0.06623366706426345, 0.016666690262465863, 0.03423317149469612, 0.02304327143120648, 0.031214508772612753, 0.045807643648171685, 0.027946448832753296, 0.049668167222889026, 0.049588148769328606, 0.024769224302769302, 0.016666667674594095, 0.05283625719167181, 0.05583979037764232, 0.045907670675591185]
gmv_epo_45_weights = [0.011111111975729409, 0.026646602204962858, 0.011111111299922065, 0.0309210193238956, 0.025007258123960922, 0.034659892275724705, 0.01111111146901072, 0.011111111747740477, 0.011111111516746397, 0.01779091912009698, 0.01984888218960698, 0.02522647335013622, 0.011180137115527101, 0.017489649124790285, 0.028191045792364738, 0.03631158995254371, 0.044444444090877805, 0.016302194182152667, 0.011111111266490585, 0.011111111595258412, 0.01111111125737019, 0.022827040798679366, 0.020587601244674014, 0.011111111503325005, 0.011111111556128327, 0.044444443639370106, 0.011111111461049875, 0.04444444402780661, 0.011111111368812045, 0.011111112285545484, 0.023035029376855205, 0.01260693000477375, 0.02165124524507929, 0.02165119066334294, 0.011286898608143595, 0.029528803525066108, 0.017506358987491956, 0.03972350417693582, 0.018956379525549516, 0.036275899230639834, 0.040688577981395774, 0.011896549529971394, 0.03665347882393091, 0.03968317677255412, 0.028087890687943526]

gmv_epo_15 = dict(zip(tickers_15, gmv_epo_15_weights))
gmv_epo_30 = dict(zip(tickers_30, gmv_epo_30_weights))
gmv_epo_45 = dict(zip(tickers_45, gmv_epo_45_weights))

### MAX Sharpe + EPO weight

In [24]:
msp_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.09539, 0.03333, 0.03333, 0.03333, 0.03707, 0.05699, 0.03897, 0.13333, 0.13333, 0.03825, 0.03333]
msp_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.0387, 0.01667, 0.01667, 0.01667, 0.06667, 0.02469, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.0443, 0.02974, 0.01667, 0.01667, 0.05967, 0.04331, 0.06667, 0.01667, 0.01667, 0.02625, 0.01667]
msp_epo_45_weights = [0.04444, 0.04379, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.0312, 0.02923, 0.04444, 0.03726, 0.04444, 0.01557, 0.01111, 0.04296, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

msp_epo_15 = dict(zip(tickers_15, msp_epo_15_weights))
msp_epo_30 = dict(zip(tickers_30, msp_epo_30_weights))
msp_epo_45 = dict(zip(tickers_45, msp_epo_45_weights))

### MAX Returns + EPO weight

In [25]:
max_return_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.13333, 0.13333, 0.03333, 0.03333]
max_return_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667]
max_return_epo_45_weights = [0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.04444, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

max_return_epo_15 = dict(zip(tickers_15, max_return_epo_15_weights))
max_return_epo_30 = dict(zip(tickers_30, max_return_epo_30_weights))
max_return_epo_45 = dict(zip(tickers_45, max_return_epo_45_weights))

In [33]:
# in-sample price data
mkt = pd.read_csv('../new_market.csv')
mkt.set_index('pricingDate', inplace=True)
mkt.index = pd.to_datetime(mkt.index)
mkt = mkt.loc[:'2023-05-01', :]
mkt

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,66.24,43.36,42.2750,102.07,58.82,26.876666,151.92,224.08,88.86,45.03,...,191.85,46.65,76.95,50.64,70.37,86.64,116.95,136.78,55.05,83.40
2018-05-02,65.91,42.24,44.1425,100.37,57.85,26.310000,150.71,221.10,88.30,44.43,...,191.03,46.54,76.80,50.59,68.88,80.20,117.46,133.12,55.19,79.90
2018-05-03,66.34,42.11,44.2225,100.28,57.93,25.983333,151.73,226.05,87.91,43.60,...,191.62,46.48,76.54,49.24,69.63,82.67,115.18,133.54,54.45,80.38
2018-05-04,67.00,42.86,45.9575,100.17,58.74,26.276666,153.44,228.51,90.27,43.86,...,192.41,46.39,76.90,49.99,70.08,82.43,116.22,134.67,55.46,81.56
2018-05-07,67.39,42.44,46.2900,99.63,59.32,26.376666,152.30,230.99,90.50,43.69,...,191.91,46.13,77.74,46.92,70.02,83.11,116.99,137.48,55.51,82.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-25,130.37,12.84,163.7700,164.90,109.97,72.590000,270.70,369.59,181.14,75.90,...,111.63,71.18,116.52,40.97,102.92,137.81,139.06,284.10,26.73,173.02
2023-04-26,133.12,12.74,163.7600,161.80,108.75,72.290000,271.21,363.06,180.66,77.03,...,109.45,69.68,115.45,40.93,100.98,138.01,139.08,280.42,26.39,172.94
2023-04-27,133.25,12.88,168.4100,148.87,109.50,73.880000,275.45,371.42,177.62,78.01,...,110.22,70.26,116.83,41.36,103.83,139.80,138.35,284.12,26.58,173.95


In [3]:
# 예시 데이터
# # 자산별 현재 가격과 포트폴리오 내 비중
# current_prices = {"Asset1": 100, "Asset2": 200, "Asset3": 300}
# current_weights = {"Asset1": 0.4, "Asset2": 0.4, "Asset3": 0.2}

# # 자산별 과거 가격 데이터 (예시)
# price_data = {
#     "Asset1": [95, 100, 105],
#     "Asset2": [195, 200, 205],
#     "Asset3": [295, 300, 305]
# }
# price_df = pd.DataFrame(price_data)

# # 수익률과 공분산 행렬 계산
# mu = expected_returns.mean_historical_return(price_df)
# S = risk_models.sample_cov(price_df)

# # 효율적인 프론티어 계산
# ef = EfficientFrontier(mu, S)
# weights = ef.max_sharpe()

# # 새로운 자산 배분
# new_weights = {k: weights[k] for k in current_weights.keys()}

# # 리밸런싱 필요 금액 계산
# portfolio_value = sum(current_prices[k] * current_weights[k] for k in current_prices)
# new_allocation = {k: portfolio_value * new_weights[k] for k in new_weights}
# rebalancing_trades = {k: new_allocation[k] - current_prices[k] * current_weights[k] for k in current_prices}

# print("새로운 자산 배분:", new_weights)
# print("리밸런싱을 위한 거래:", rebalancing_trades)